In [ ]:
# setup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# define style do seaborn
sns.set_style('whitegrid')

In [ ]:
# carrega o dataset tratado
df_br = pd.read_csv('../data/brasileirao_2014_2024_clean.csv', index_col=0)

# exibe informações sobre o dataframe
df_br.info()

In [ ]:
# define o vencedor com base nos gols e atribui "empate" se os gols forem iguais
df_br['vencedor'] = np.select(
    [df_br['gols_mandante'] > df_br['gols_visitante'], df_br['gols_mandante'] < df_br['gols_visitante']],
    [df_br['time_mandante'], df_br['time_visitante']],
    default='Empate'
)

# calcula a quantidade de pontos feitos como mandante/visitante
# vitória = 3 pontos; empate = 1 ponto; derrota = 0 pontos
df_br['pontos_mandante'] = np.select(
    [df_br['vencedor'] == df_br['time_mandante'], df_br['vencedor'] == 'Empate'],
    [3, 1],
    default=0
)

df_br['pontos_visitante'] = np.select(
    [df_br['vencedor'] == df_br['time_visitante'], df_br['vencedor'] == 'Empate'],
    [3, 1],
    default=0
)

In [ ]:
# cálculo da taxa de aproveitamento de pontos - times mandantes/casa
aprov_mandante = (
    df_br
    .groupby(['ano_campeonato', 'time_mandante'])
    .agg(
        partidas_mandante=('pontos_mandante', 'size'),
        pontos_mandante=('pontos_mandante', 'sum')        
    )
    .assign(aproveitamento_mandante=lambda df: df['pontos_mandante'] / (df['partidas_mandante'] * 3))
    .rename_axis(['ano_campeonato', 'time'])
    .reset_index()
)

# cálculo da taxa de aproveitamento de pontos - times visitantes/fora
aprov_visitante = (
    df_br
    .groupby(['ano_campeonato', 'time_visitante'])
    .agg(
        partidas_visitante=('pontos_visitante', 'size'),
        pontos_visitante=('pontos_visitante', 'sum')        
    )
    .assign(aproveitamento_visitante=lambda df: df['pontos_visitante'] / (df['partidas_visitante'] * 3))
    .rename_axis(['ano_campeonato', 'time'])
    .reset_index()
)

# junção das bases de aproveitamento de pontos
df_aproveitamento = (
    aprov_mandante
    .merge(aprov_visitante, on=['ano_campeonato', 'time'])
    .assign(desvio_aproveitamento=lambda df: df['aproveitamento_mandante'] - df['aproveitamento_visitante'])
    .sort_values(by=['ano_campeonato', 'time'], ascending=True)
)

In [ ]:
# check se há redução no efeito mandante durante a pandemia
# resposta curta: queda da diferença mandante - visitante acontece já a partir de 2019
desvio_temporada = (
    df_aproveitamento
    .groupby('ano_campeonato')
    .agg(desvio_aproveitamento=('desvio_aproveitamento', 'mean'))
    .reset_index()
)

plt.figure(figsize=(9, 5))

sns.lineplot(
    data=desvio_temporada,
    x='ano_campeonato',
    y='desvio_aproveitamento',
    marker='o',
    color='slateblue'
)

plt.axvspan(2019.5, 2021.5, color='red', alpha=0.2, label='Pandemia (estádios vazios)')
plt.legend()

plt.xlabel('Temporada')
plt.xticks(desvio_temporada['ano_campeonato'], rotation=45)
plt.ylabel('Desvio médio de aproveitamento\n(mandante - visitante)')
plt.ylim([0, 0.4])

plt.tight_layout()
plt.show()

In [ ]:
# identifica partidas sem público nos anos de pandemia (2020 e 2021) 
df_br['pandemia'] = df_br['ano_campeonato'].isin([2020, 2021]).astype(int)

# define variável lotação dos estádios (público real / capacidade máxima)
df_br['lotacao_estadio'] = (df_br['publico'] / df_br['publico_max']).clip(upper=1)

# distribuição da variável de lotação dos estádios vs dummy de pandemia
plt.figure(figsize=(8, 6))

sns.histplot(
    data = df_br,
    x='lotacao_estadio', 
    bins=30,
    hue='pandemia',
    palette='coolwarm'
)

plt.legend(title='Pandemia', labels=['Sim', 'Não'])
plt.xlabel('Lotação dos estádios\n(público real / capacidade máxima)')
plt.ylabel('Número de partidas')

plt.tight_layout()
plt.show()

In [ ]:
# define bins para categorizar a lotação dos estádios
df_br['lotacao_estadio_cat'] = pd.cut(df_br['lotacao_estadio'], bins=20)

# define variável que identifica vitória do mandante
df_br['vitoria_mandante'] = (df_br['pontos_mandante'] == 3).astype(int)

# dataframe agrupado pelos bins da lotação contendo:
# a) número total de partidas; b) lotação média dos jogos; c) probabilidade de vitória do mandante
lotacao_vitoria = (
    df_br
    .groupby('lotacao_estadio_cat', observed=False)
    .agg(
        n_partidas=('vitoria_mandante', 'count'),
        lotacao_media=('lotacao_estadio', 'mean'),
        prob_vitoria_mandante=('vitoria_mandante', 'mean')
    )
    .reset_index()
)

# relação entre lotação dos estádios e probabilidade de vitória dos mandantes: efeito tímido
plt.figure(figsize=(8, 6))

sns.regplot(
    data=lotacao_vitoria,
    x='lotacao_media',
    y='prob_vitoria_mandante',
    color='navy',
    scatter_kws={'alpha': 0.6},
    line_kws={'color': 'black'}
)

plt.xlabel('Lotação dos estádios')
plt.ylabel('Probabilida de vitória do mandante')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()


In [ ]:
# identifica terço do campeonato a partir das rodadas
df_br['terco_campeonato'] = pd.cut(
    x=df_br['rodada'],
    bins=[0, 13, 25, 38],
    labels=['1º terço', '2º terço', '3º terço'],
    right=True
)

# identifica partidas de clássicos
classicos = [
    # paulistas
    frozenset(['Palmeiras', 'Corinthians']),
    frozenset(['Palmeiras', 'São Paulo']),
    frozenset(['Palmeiras', 'Santos']),
    frozenset(['Corinthians', 'São Paulo']),
    frozenset(['Corinthians', 'Santos']),
    frozenset(['São Paulo', 'Santos']),

    # cariocas
    frozenset(['Flamengo', 'Fluminense']),
    frozenset(['Flamengo', 'Vasco da Gama']),
    frozenset(['Flamengo', 'Botafogo']),
    frozenset(['Fluminense', 'Vasco da Gama']),
    frozenset(['Fluminense', 'Botafogo']),
    frozenset(['Vasco da Gama', 'Botafogo']),

    # gaúcho
    frozenset(['Grêmio', 'Internacional']),

    # mineiros
    frozenset(['Atlético-MG', 'Cruzeiro']),
    frozenset(['Atlético-MG', 'América-MG']),
    frozenset(['Cruzeiro', 'América-MG']),

    # paranaense
    frozenset(['Atlético-PR', 'Coritiba']),

    # baianos
    frozenset(['Bahia', 'Vitória']),

    # cearenses
    frozenset(['Ceará SC', 'Fortaleza']),

    # goianos
    frozenset(['Atlético-GO', 'Goiás']),

    # pernambucanos
    frozenset(['Sport Recife', 'Santa Cruz']),
]

confrontos = df_br[['time_mandante', 'time_visitante']].agg(frozenset, axis=1)

df_br['classico'] = confrontos.isin(classicos).astype(int)